# `lynx` Tutorial

A quick overview covering how to use `lynx` and it can be utilized to take advantage of relational data.

# Overview

`lynx` introduces a new tabular structure, `Table`, that retains relational join information. This is achieved by modeling a `Table` as a list of tabular blocks that are lazily joined. A lazy join in this context just means that the join is only represented by a rows index mappings and the rows of the tabular block are not mapped to the join mappings unless necessary.

At a low level, `lynx.Table` blocks are modeled using `pandas.DataFrames` and `sparse.csr_matrix`es accompanied by a block name and row index mapping. The table can then be transformed to different data formats specific to desired algorithms.

This enables accessibility to machine learning algorithms that can learn on relational data without materializing joins. One example of this is `libFM`'s block structure algorithms.

In addition, `lynx` provides many common data wrangling operations on `lynx.Tables` such as one hot encoding and modeling feature interactions.

Let's jump into some code examples now! We will run through an example of predicting movie rating scores for users. The example data is based off of the example data presented in "[Scaling Factorization Machines to Relational Data](https://www.vldb.org/pvldb/vol6/p337-rendle.pdf)".

# Code Example

## Create Example Data

First, let's create the base `pandas.DataFrames` that will be used to demonstrate usage of `lynx`.

In [1]:
import pandas as pd

#### Movie Ratings Table

Rows of ratings that users gave each movie they've watched.

In [2]:
ratings = pd.DataFrame(
    [
        ["Alice", "TI", 5, "2012-09-01"],
        ["Alice", "NH", 3, "2012-09-12"],
        ["Alice", "SW", 1, "2012-09-15"],
        ["Bob", "SW", 4, "2012-09-02"],
        ["Bob", "ST", 5, "2012-10-07"],
        ["Charlie", "TI", 1, "2012-09-05"],
        ["Charlie", "SW", 1, "2012-09-05"],
    ],
    columns=["user_id", "movie_id", "score", "date"]
)
ratings

,user_id,movie_id,score,date
0,Alice,TI,5,2012-09-01
1,Alice,NH,3,2012-09-12
2,Alice,SW,1,2012-09-15
3,Bob,SW,4,2012-09-02
4,Bob,ST,5,2012-10-07
5,Charlie,TI,1,2012-09-05
6,Charlie,SW,1,2012-09-05


#### Users Table

User profile metadata, i.e. id, gender, and age.

In [3]:
users = pd.DataFrame(
    [
        ["Alice", "F", 30],
        ["Bob", "M", 25],
        ["Charlie", "M", 28]
    ],
    columns=["id", "gender", "age"]
)
users

,id,gender,age
0,Alice,F,30
1,Bob,M,25
2,Charlie,M,28


#### Movies Table

Movie metadata, i.e. movie genre.

In [4]:
movies = pd.DataFrame(
    [
        ["TI", ["Action", "Romance"]],
        ["SW", ["Science Fiction"]],
        ["ST", ["Science Fiction"]],
        ["NH", ["Romance", "Comedy"]]
    ],
    columns=["id", "genres"]
)
movies

,id,genres
0,TI,"[Action, Romance]"
1,SW,[Science Fiction]
2,ST,[Science Fiction]
3,NH,"[Romance, Comedy]"


#### Friends Table

Network of which users are friends with each other.

In [5]:
friends = pd.DataFrame(
    [
        ["Alice", "Charlie"],
        ["Bob", "Charlie"],
        ["Charlie", "Alice"],
        ["Charlie", "Bob"]
    ],
    columns=["user_id", "friend_id"]
)
friends

,user_id,friend_id
0,Alice,Charlie
1,Bob,Charlie
2,Charlie,Alice
3,Charlie,Bob


## Using `lynx`

First, let's import `lynx`.

In [6]:
import lynx as lx

To use a `lynx.Table`, we can provide a `pandas.DataFrame` or `sparse.csr_matrix`
as an initial tabular block with a name for the block. A `lynx.Table` can be
transformed into a `pandas.DataFrame` at any time.

In [7]:
ratings_table = lx.Table(ratings, "ratings")
ratings_table.to_dataframe()

,user_id,movie_id,score,date
0,Alice,TI,5,2012-09-01
1,Alice,NH,3,2012-09-12
2,Alice,SW,1,2012-09-15
3,Bob,SW,4,2012-09-02
4,Bob,ST,5,2012-10-07
5,Charlie,TI,1,2012-09-05
6,Charlie,SW,1,2012-09-05


`lynx.Table`s provide methods for popular data wrangling operations.

One-hot encoding can be achieved with `onehot()`. Note that the encoding
operations append the encoded data as a block at the right end of the 
`lynx.Table` and, by default, drop the provided column though this can be 
controlled with the `drop` argument.

In [8]:
users_table = (
    lx.Table(users, "users")
    .onehot("gender")
)
users_table.to_dataframe()

,id,age,0,1
0,Alice,30,1,0
1,Bob,25,0,1
2,Charlie,28,0,1


Many-hot encoding can be achieved with `manyhot()`.

In [9]:
movies_table = (
    lx.Table(movies, "movies")
    .manyhot("genres")
)
movies_table.to_dataframe()

,id,0,1,2,3
0,TI,0.5,0.0,0.5,0.0
1,SW,0.0,0.0,0.0,1.0
2,ST,0.0,0.0,0.0,1.0
3,NH,0.0,0.5,0.5,0.0


An interactions matrix can be modeled with `model_interactions()`. This
operations is similar to a `manyhot()` applied on exploded data. Here, we are
creating a matrix to model users and all of the friends that they have.

`model_interactions()` does not drop any columns.

In [10]:
friends_table = (
    lx.Table(friends, "friends")
    .model_interactions("user_id", "friend_id")
)
friends_table.to_dataframe()

,user_id,friend_id,0,1,2
0,Alice,Charlie,0.0,0.0,1.0
1,Bob,Charlie,0.0,0.0,1.0
2,Charlie,Alice,0.5,0.5,0.0
3,Charlie,Bob,0.5,0.5,0.0


Now, to start mixing in the relational nature of this data, we can join tables 
on specified columns using `merge()`. `merge()` applies a database-style inner 
join.

In [11]:
merged_table = (
    ratings_table
    .merge(users_table, left_on="user_id", right_on="id")
    .merge(movies_table, left_on="movie_id", right_on="id")
    .merge(friends_table, left_on="user_id", right_on="user_id")
)
merged_table.to_dataframe()

,user_id,movie_id,score,date,age,0,1,0,1,2,3,friend_id,0,1,2
0,Alice,TI,5,2012-09-01,30,1,0,0.5,0.0,0.5,0.0,Charlie,0.0,0.0,1.0
1,Alice,NH,3,2012-09-12,30,1,0,0.0,0.5,0.5,0.0,Charlie,0.0,0.0,1.0
2,Alice,SW,1,2012-09-15,30,1,0,0.0,0.0,0.0,1.0,Charlie,0.0,0.0,1.0
3,Charlie,TI,1,2012-09-05,28,0,1,0.5,0.0,0.5,0.0,Alice,0.5,0.5,0.0
4,Charlie,TI,1,2012-09-05,28,0,1,0.5,0.0,0.5,0.0,Bob,0.5,0.5,0.0
5,Charlie,SW,1,2012-09-05,28,0,1,0.0,0.0,0.0,1.0,Alice,0.5,0.5,0.0
6,Charlie,SW,1,2012-09-05,28,0,1,0.0,0.0,0.0,1.0,Bob,0.5,0.5,0.0
7,Bob,SW,4,2012-09-02,25,0,1,0.0,0.0,0.0,1.0,Charlie,0.0,0.0,1.0
8,Bob,ST,5,2012-10-07,25,0,1,0.0,0.0,0.0,1.0,Charlie,0.0,0.0,1.0


To drop unwanted columns, we can use `drop`.

In [12]:
merged_table = (
    merged_table
    .drop(["date", "friend_id"])
)
merged_table.to_dataframe()

,user_id,movie_id,score,age,0,1,0,1,2,3,0,1,2
0,Alice,TI,5,30,1,0,0.5,0.0,0.5,0.0,0.0,0.0,1.0
1,Alice,NH,3,30,1,0,0.0,0.5,0.5,0.0,0.0,0.0,1.0
2,Alice,SW,1,30,1,0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,Charlie,TI,1,28,0,1,0.5,0.0,0.5,0.0,0.5,0.5,0.0
4,Charlie,TI,1,28,0,1,0.5,0.0,0.5,0.0,0.5,0.5,0.0
5,Charlie,SW,1,28,0,1,0.0,0.0,0.0,1.0,0.5,0.5,0.0
6,Charlie,SW,1,28,0,1,0.0,0.0,0.0,1.0,0.5,0.5,0.0
7,Bob,SW,4,25,0,1,0.0,0.0,0.0,1.0,0.0,0.0,1.0
8,Bob,ST,5,25,0,1,0.0,0.0,0.0,1.0,0.0,0.0,1.0


The `lynx.Table` operations are still applicable after merges. Let's 
additionally model the interactions between users and all of the movies that 
they've watched.

In [13]:
merged_table = merged_table.model_interactions("user_id", "movie_id")
merged_table.to_dataframe()

,user_id,movie_id,score,age,0,1,0,1,2,3,0,1,2,0,1,2,3
0,Alice,TI,5,30,1,0,0.5,0.0,0.5,0.0,0.0,0.0,1.0,0.333333,0.0,0.333333,0.333333
1,Alice,NH,3,30,1,0,0.0,0.5,0.5,0.0,0.0,0.0,1.0,0.333333,0.0,0.333333,0.333333
2,Alice,SW,1,30,1,0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.333333,0.0,0.333333,0.333333
3,Charlie,TI,1,28,0,1,0.5,0.0,0.5,0.0,0.5,0.5,0.0,0.000000,0.5,0.500000,0.000000
4,Charlie,TI,1,28,0,1,0.5,0.0,0.5,0.0,0.5,0.5,0.0,0.000000,0.5,0.500000,0.000000
5,Charlie,SW,1,28,0,1,0.0,0.0,0.0,1.0,0.5,0.5,0.0,0.000000,0.5,0.500000,0.000000
6,Charlie,SW,1,28,0,1,0.0,0.0,0.0,1.0,0.5,0.5,0.0,0.000000,0.5,0.500000,0.000000
7,Bob,SW,4,25,0,1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.000000,0.0,0.500000,0.500000
8,Bob,ST,5,25,0,1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.000000,0.0,0.500000,0.500000


To wrap up our feature engineering, let's one-hot encode the user and movie id 
columns. We have now encoded all categorical variables and after encoding, they
were appended to the right end of the `lynx.Table`.

In [14]:
merged_table = (
    merged_table
    .onehot("user_id")
    .onehot("movie_id")
)
merged_table.to_dataframe()

,score,age,0,1,0,1,2,3,0,1,...,1,2,3,0,1,2,0,1,2,3
0,5,30,1,0,0.5,0.0,0.5,0.0,0.0,0.0,...,0.0,0.333333,0.333333,1,0,0,0,0,0,1
1,3,30,1,0,0.0,0.5,0.5,0.0,0.0,0.0,...,0.0,0.333333,0.333333,1,0,0,1,0,0,0
2,1,30,1,0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.333333,0.333333,1,0,0,0,0,1,0
3,1,28,0,1,0.5,0.0,0.5,0.0,0.5,0.5,...,0.5,0.500000,0.000000,0,0,1,0,0,0,1
4,1,28,0,1,0.5,0.0,0.5,0.0,0.5,0.5,...,0.5,0.500000,0.000000,0,0,1,0,0,0,1
5,1,28,0,1,0.0,0.0,0.0,1.0,0.5,0.5,...,0.5,0.500000,0.000000,0,0,1,0,0,1,0
6,1,28,0,1,0.0,0.0,0.0,1.0,0.5,0.5,...,0.5,0.500000,0.000000,0,0,1,0,0,1,0
7,4,25,0,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.500000,0.500000,0,1,0,0,0,1,0
8,5,25,0,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.500000,0.500000,0,1,0,0,1,0,0


When all features of a `lynx.Table` are numerical, we can also convert it to a
`sparse.csr_matrix`.

In [15]:
merged_table.to_csr_matrix()

<9x22 sparse matrix of type '<class 'numpy.float64'>'
	with 92 stored elements in Compressed Sparse Row format>

Now that we have finished wrangling our relational dataset, let's use the
engineered `lynx.Table` in an experiment!

## Experiments

To set up our experiments, let's first extract the target variables and split 
our data into train and test sets. `lynx.Table` provides the `pop()` method for
extracting a specified column. `lynx.Table` is also compatible with the popular
`sklearn.model_selection.train_test_split()` method for splitting datasets.

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

seed = 0

y = merged_table.pop("score")
X_train, X_test, y_train, y_test = train_test_split(
    merged_table, y,
    train_size=0.8,
    test_size=0.2,
    random_state=seed
)

display("TRAIN DATASET")
display(X_train.to_dataframe())
display(y_train)
display("TEST DATASET")
display(X_test.to_dataframe())
display(y_test)

'TRAIN DATASET'

,age,0,1,0,1,2,3,0,1,2,...,1,2,3,0,1,2,0,1,2,3
0,30,1,0,0.0,0.5,0.5,0.0,0.0,0.0,1.0,...,0.0,0.333333,0.333333,1,0,0,1,0,0,0
1,28,0,1,0.5,0.0,0.5,0.0,0.5,0.5,0.0,...,0.5,0.500000,0.000000,0,0,1,0,0,0,1
2,25,0,1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.500000,0.500000,0,1,0,0,1,0,0
3,28,0,1,0.0,0.0,0.0,1.0,0.5,0.5,0.0,...,0.5,0.500000,0.000000,0,0,1,0,0,1,0
4,28,0,1,0.5,0.0,0.5,0.0,0.5,0.5,0.0,...,0.5,0.500000,0.000000,0,0,1,0,0,0,1
5,30,1,0,0.5,0.0,0.5,0.0,0.0,0.0,1.0,...,0.0,0.333333,0.333333,1,0,0,0,0,0,1
6,28,0,1,0.0,0.0,0.0,1.0,0.5,0.5,0.0,...,0.5,0.500000,0.000000,0,0,1,0,0,1,0


1    3
4    1
8    5
6    1
3    1
0    5
5    1
Name: score, dtype: int64

'TEST DATASET'

,age,0,1,0,1,2,3,0,1,2,...,1,2,3,0,1,2,0,1,2,3
0,25,0,1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.500000,0.500000,0,1,0,0,0,1,0
1,30,1,0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.333333,0.333333,1,0,0,0,0,1,0


7    4
2    1
Name: score, dtype: int64

### libFM

The first experiment we'll run is a basic MCMC regression task where no
relational features are taken advantage of.

In [17]:
from lynx.libfm import mcmc

fm= mcmc.FMRegression(iter_num=100, seed=seed)
predictions = fm.fit_predict(X_train, y_train, X_test)
fm.flush() # Remove temporary libFM files

mean_squared_error(predictions, y_test, squared=False)

2.147669192031212

Now, let's use `libFM`'s block structure MCMC regression task which takes
advantage of relational features.

In [18]:
from lynx.libfm.bs import mcmc

mcmc_r = mcmc.FMRegression(iter_num=100, seed=seed)
predictions = mcmc_r.fit_predict(X_train, y_train, X_test)
mcmc_r.flush() # Remove temporary libFM files

mean_squared_error(predictions, y_test, squared=False)

2.5701892730692033

## More Details

This dummy example doesn't properly illustrate the advantages of using `libFM`'s
block structure. The RMSE shouldn't actually be that different, but because of
the dataset size being poor, the predictions are quite varied. The real
advantage of the relational algorithm is that its training time scales linearly
with the number of non-zero elements in the dataset *before* joins are 
materialized. The basic learning algorithm's runtime, however, scales linearly
with the total number of non-zero elements in the dataset.

To illustrate this, we can look at how `lynx.Table` stores each tabular block.
First, let's get all of the names of each block and the shape of them before
joins have been materialized.

In [19]:
display(merged_table.block_names)
display(merged_table.block_shapes)

['users',
 'gender_onehot',
 'genres_manyhot',
 'user_id_friend_id_interactions',
 'user_id_movie_id_interactions',
 'user_id_onehot',
 'movie_id_onehot']

{'users': (3, 1),
 'gender_onehot': (2, 2),
 'genres_manyhot': (3, 4),
 'user_id_friend_id_interactions': (3, 3),
 'user_id_movie_id_interactions': (3, 4),
 'user_id_onehot': (3, 3),
 'movie_id_onehot': (4, 4)}

Now, let's look closer at the `user_id_movie_id_interactions` block. It's
data before joining has 3 rows. After materializing the join, however, the block
has 9 rows, where multiple rows are duplicated.

In [20]:
(
    merged_table
    .get_block("user_id_movie_id_interactions")
    .get_block_csr_matrix()
    .todense()
)

matrix([[0.33333333, 0.        , 0.33333333, 0.33333333],
        [0.        , 0.5       , 0.5       , 0.        ],
        [0.        , 0.        , 0.5       , 0.5       ]])

In [21]:
(
    merged_table
    .get_block("user_id_movie_id_interactions")
    .to_csr_matrix()
    .todense()
)

matrix([[0.33333333, 0.        , 0.33333333, 0.33333333],
        [0.33333333, 0.        , 0.33333333, 0.33333333],
        [0.33333333, 0.        , 0.33333333, 0.33333333],
        [0.        , 0.5       , 0.5       , 0.        ],
        [0.        , 0.5       , 0.5       , 0.        ],
        [0.        , 0.5       , 0.5       , 0.        ],
        [0.        , 0.5       , 0.5       , 0.        ],
        [0.        , 0.        , 0.5       , 0.5       ],
        [0.        , 0.        , 0.5       , 0.5       ]])

With this data duplication in mind, the number of non-zero elements before
joining and after joining are as follows.

In [22]:
print(f"Joined NNZ: {merged_table.nnz}")
print(f"Non-Joined NNZ: {merged_table.block_nnz}")

Joined NNZ: 83
Non-Joined NNZ: 28


Now, if learning algorithms were to scale linearly with the number of non-zero
numbers, taking advantage of the non-joined NNZ would yield much better
performance. This is what block structure in `libFM` takes advantage of and that
`lynx.Table` makes accessible.

`lynx` also provides a method for writing out the SVMLight data format that 
`libFM` requires through `lynx.write`, e.g.

- `lynx.write.libfm()`
- `lynx.write.libfm.bs()`

which should enable more exploration of `libFM`.